In [3]:
import numpy as np
import pandas as pd

In [4]:
train = pd.read_csv('/home/bogan/Загрузки/train.csv')
test = pd.read_csv('/home/bogan/Загрузки/test.csv')

In [ ]:
train = train.sample(frac=1)

In [ ]:
train[train == 'na'] = np.nan

In [ ]:
test[test == "na"] = np.nan

In [ ]:
data = train.iloc[:, 1:43]

In [ ]:
y = train.iloc[:, 43]

In [ ]:
def norm_to_gauss(df):
    return (df - df.mean()) * 1.0 / (df.var())

In [ ]:
data = data.apply(np.float64)

In [ ]:
data = data.apply(norm_to_gauss)

In [ ]:
data_test = test.iloc[:, 1:43]

In [ ]:
data_test = data_test.apply(np.float32)

In [ ]:
data_test = data_test.apply(norm_to_gauss)

In [ ]:
import sklearn

In [1]:
from sklearn.impute import MICEImputer

In [2]:
scaler = MICEImputer(n_imputations=70)

In [ ]:
scaler.fit(data)

In [ ]:
data = scaler.transform(data)

In [ ]:
data_test = scaler.transform(data_test)

In [ ]:
def argmax(preds):
    return np.asarray([np.argmax(line) for line in preds])

In [ ]:
from keras.utils.np_utils import to_categorical   

cl = to_categorical(y, num_classes=3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
model = Sequential()

model.add(Dense(128, init='uniform', input_dim=42, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(64, init='uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(12, init='uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(3, init='uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(data, cl, epochs=25, batch_size=100)
predictions = model.predict(data_train)

In [ ]:
pr = argmax(predictions)

In [ ]:
d = {'objid': train.iloc[:, 0], 'prediction': pr}
df = pd.DataFrame(data=d)

In [9]:
df.to_csv('predictions.csv', encoding='utf-8', index=False)